In [ ]:
# Load libraries

from __future__ import absolute_import, division, print_function, unicode_literals

import stellargraph as sg
from stellargraph.mapper import FullBatchNodeGenerator, KGTripleGenerator
from stellargraph.layer import GCN, ComplEx
from stellargraph import StellarDiGraph


try:
    sg.utils.validate_notebook_version("1.2.1")
except AttributeError:
    raise ValueError(
        f"Requires StellarGraph version 1.2.1, but version {sg.__version__} is detected."
    ) from None

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks, regularizers, optimizers, losses, metrics, Model, backend as bk
from tensorflow.keras.layers import Dense, Flatten, Conv2D
print("TensorFlow version: ", tf.__version__)

import snap
import scipy.sparse as sp
import numpy as np
from sklearn import preprocessing

import collections
from collections import namedtuple
import unittest
import os

import pandas as pd
import seaborn as sns; sns.set()
import matplotlib.image as mpimg
import scipy
import pickle as pkl
import random
import time

from Bio.PDB import PDBParser
from Bio.PDB.Polypeptide import PPBuilder

from sklearn import preprocessing, model_selection
from sklearn import metrics as skm

from IPython.display import display, HTML
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option("display.max_rows", 100, "display.max_columns", 100)

TensorFlow version:  2.6.0


In [ ]:
%cd /content/drive/MyDrive/_Projects/epistasis/cs229_project

/content/drive/MyDrive/_Projects/epistasis/cs229_project


In [ ]:
# Load DMS from csv files

SM = pd.read_csv('olson_singles.csv')
DM = pd.read_csv('olson_multis.csv')

SM.rename(columns={'WT amino acid':'m1_wt_aa','Position':'m1_pos','Mutation':'m1_mt_aa','Input Count':'input_count','Selection Count':'selection_count'},inplace=True)
# SM.head()

DM.rename(columns={'Mut1 WT amino acid':'m1_wt_aa','Mut1 Position':'m1_pos','Mut1 Mutation':'m1_mt_aa',
                   'Mut2 WT amino acid':'m2_wt_aa','Mut2 Position':'m2_pos','Mut2 Mutation':'m2_mt_aa',
                   'Input Count':'input_count_dm','Selection Count':'selection_count_dm',
                   'Mut1 Fitness':'mut1_fitness','Mut2 Fitness':'mut2_fitness'},inplace=True)

DM = DM.sample(frac=1)
DM = DM.reset_index(drop=True)

DM = DM.drop_duplicates(subset=['m1_wt_aa','m1_pos','m1_mt_aa','m2_wt_aa','m2_pos','m2_mt_aa'])

DM['dmlabel'] = DM.apply(lambda x: x.m1_mt_aa + x.m2_mt_aa, axis=1)

DM

,m1_wt_aa,m1_pos,m1_mt_aa,m2_wt_aa,m2_pos,m2_mt_aa,input_count_dm,selection_count_dm,mut1_fitness,mut2_fitness,dmlabel
0,K,4,I,L,5,I,799,235,0.965,0.206,II
1,G,38,I,N,37,G,157,1,0.207,0.379,IG
2,V,39,N,W,43,G,1012,3,0.010,0.005,NG
3,K,31,N,Y,45,R,343,5,0.004,0.048,NR
4,K,31,I,Y,33,S,2251,9,0.004,0.182,IS
...,...,...,...,...,...,...,...,...,...,...,...
535912,D,36,A,Y,33,P,537,10,1.660,0.008,AP
535913,D,36,C,L,7,W,74,17,1.506,0.059,CW
535914,D,46,V,E,27,N,685,6,1.281,0.003,VN
535915,A,20,W,A,48,N,91,73,0.395,1.254,WN


In [ ]:
# CONSIDER ADJUSTING AA INDICES

In [ ]:
# We are going to calculate DMF directly from the input and selection counts
# And then shift it up by 1 to put it on the same scale #####


# Calculate DMF
scale = 1
shift = 1 + 0.002 # keep everything positive with same min as SMFs

DM['dmf'] = DM.apply(lambda x: (x.selection_count_dm - x.input_count_dm)/x.input_count_dm*scale + shift, axis=1)
DM.rename(columns={'mut1_fitness':'smf1','mut2_fitness':'smf2'},inplace=True)
DM = DM[['m1_wt_aa','m1_pos','m1_mt_aa','m2_wt_aa','m2_pos','m2_mt_aa','smf1','smf2','dmf','dmlabel']]

# Calculate epistasis

# using logarithmic definition ####
DM['epistasis'] = DM.apply(lambda x: np.log(x.dmf) - np.log(x.smf1) - np.log(x.smf2), axis=1)
DM

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,m1_wt_aa,m1_pos,m1_mt_aa,m2_wt_aa,m2_pos,m2_mt_aa,smf1,smf2,dmf,dmlabel,epistasis
0,K,4,I,L,5,I,0.965,0.206,0.296118,II,0.398508
1,G,38,I,N,37,G,0.207,0.379,0.008369,IG,-2.237914
2,V,39,N,W,43,G,0.010,0.005,0.004964,NG,4.598030
3,K,31,N,Y,45,R,0.004,0.048,0.016577,NR,4.458292
4,K,31,I,Y,33,S,0.004,0.182,0.005998,IS,2.108917
...,...,...,...,...,...,...,...,...,...,...,...
535912,D,36,A,Y,33,P,1.660,0.008,0.020622,AP,0.440098
535913,D,36,C,L,7,W,1.506,0.059,0.231730,CW,0.958577
535914,D,46,V,E,27,N,1.281,0.003,0.010759,VN,1.029501
535915,A,20,W,A,48,N,0.395,1.254,0.804198,WN,0.484621


In [ ]:
seq = ['-' for i in range(56)]
seq[0] = 'M'
for i in range(len(DM)):
     pos = DM.m1_pos.loc[i]-1
     wt = DM.m1_wt_aa.loc[i]
     if seq[pos] == '-':
         seq[pos] = wt

for i in range(len(DM)):
     pos = DM.m2_pos.loc[i]-1
     wt = DM.m2_wt_aa.loc[i]
     if seq[pos] == '-':
         seq[pos] = wt

seq = ''.join(seq)

pdbid = '1PGA'

parser = PDBParser()
structure = parser.get_structure(pdbid, '/content/drive/My Drive/' + pdbid + '.pdb')

model = structure[0]
chain = model['A']

ppb = PPBuilder()
sequence = str(ppb.build_peptides(chain)[0].get_sequence())

print(seq)
print(sequence)

MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE
MTYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE


In [ ]:
mutseqs = []
for i in range(len(DM)):

    m1_pos = DM.m1_pos.loc[i]-1
    m2_pos = DM.m2_pos.loc[i]-1
    m1_mut = DM.m1_mt_aa.loc[i]
    m2_mut = DM.m2_mt_aa.loc[i]
    e = DM.epistasis.loc[i]

    if DM.m1_wt_aa.loc[i] != seq[m1_pos]:
        print('Alignment error')
        break

    mutseq = [x for x in seq]
    mutseq[m1_pos] = m1_mut
    mutseq[m2_pos] = m2_mut
    mutseq = ''.join(mutseq)

    title = str(m1_pos+1)+'_'+str(m1_mut)+'_'+str(m2_pos+1)+'_'+str(m2_mut)+'_'+str(e)

    if i % 10000 == 0:
        print(i)

    mutseqs.append((title, mutseq))

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000


In [ ]:
ofile = open('gb1_dm_sequences.txt', 'w')
for i in range(len(mutseqs)):
    ofile.write('>' + mutseqs[i][0] + '\n' + mutseqs[i][1] + '\n')
ofile.close()